# 🧹 PACE Bootcamp Comprehensive Cleanup

This notebook provides a comprehensive cleanup process for all resources created during the PACE Bootcamp.

## Overview

This cleanup process will remove:
- **Tagged Resources**: All resources tagged with `app:pace_bootcamp`
- **AgentCore Memory**: Memory resources and stored data
- **AgentCore Gateway**: Gateway targets and endpoints
- **AgentCore Runtime**: Agent runtime instances
- **ECR Repositories**: Container images
- **S3 Vectors**: Vector buckets and indexes
- **Secrets Manager**: Secrets and credentials
- **CodeBuild**: Build projects
- **Security Resources**: IAM roles, Cognito pools
- **Observability**: CloudWatch logs
- **Local Files**: Generated configuration files

⚠️ **Important**: This cleanup is irreversible. Make sure you have saved any important data before proceeding.

---

## Step 1: Import Required Dependencies

In [ ]:
from cleanup_tagged_resources import get_tagged_resources, delete_resources
import boto3
import sys
import os

# Add AgentCore lab helpers to path if they exist
agentcore_path = os.path.join(os.path.dirname(os.getcwd()), 'agentcore')
if os.path.exists(agentcore_path) and agentcore_path not in sys.path:
    sys.path.insert(0, agentcore_path)

# Try to import AgentCore utilities (optional)
try:
    from lab_helpers.utils import (
        delete_agentcore_runtime_execution_role,
        delete_ssm_parameter,
        cleanup_cognito_resources,
        get_customer_support_secret,
        delete_customer_support_secret,
        agentcore_memory_cleanup,
        gateway_target_cleanup,
        runtime_resource_cleanup,
        delete_observability_resources,
        local_file_cleanup
    )
    AGENTCORE_AVAILABLE = True
    print("✅ AgentCore utilities loaded")
except ImportError:
    AGENTCORE_AVAILABLE = False
    print("ℹ️  AgentCore utilities not available (skipping AgentCore-specific cleanup)")

print("✅ Dependencies imported successfully")

## Step 2: Delete Tagged Resources

Remove all resources tagged with `app:pace_bootcamp`

In [ ]:
resources = get_tagged_resources("app", "pace_bootcamp")
for r in resources:
    print(f"Found {r}")

In [ ]:
delete_resources(resources)

## Step 3: Handle Deletion of Untagged Resources

Initialize AWS clients for resource-specific cleanup

In [ ]:
agentcore_control_client = boto3.client('bedrock-agentcore-control')
ecr_client = boto3.client('ecr')
secrets_client = boto3.client('secretsmanager')
codebuild = boto3.client('codebuild')
s3vectors_client = boto3.client('s3vectors')
logs_client = boto3.client('logs')

print("✅ AWS clients initialized")

### Step 3.1: Clean Up AgentCore Memory Resources

Remove AgentCore Memory resources and associated data (if AgentCore labs were completed)

In [ ]:
if AGENTCORE_AVAILABLE:
    print("🧠 Starting AgentCore Memory cleanup...")
    try:
        agentcore_memory_cleanup()
        print("✅ AgentCore Memory cleanup completed")
    except Exception as e:
        print(f"⚠️  Error during memory cleanup: {e}")
else:
    print("ℹ️  Skipping AgentCore Memory cleanup (utilities not available)")

### Step 3.2: Clean Up AgentCore Gateway Resources

Remove Gateway targets and endpoints (if AgentCore labs were completed)

In [ ]:
if AGENTCORE_AVAILABLE:
    print("⚙️  Starting AgentCore Gateway cleanup...")
    try:
        gateway_target_cleanup()
        print("✅ AgentCore Gateway cleanup completed")
    except Exception as e:
        print(f"⚠️  Error during gateway cleanup: {e}")
else:
    print("ℹ️  Skipping AgentCore Gateway cleanup (utilities not available)")

### Step 3.3: Clean Up AgentCore Runtime Resources

Remove AgentCore Runtime instances and ECR repositories (if AgentCore labs were completed)

In [ ]:
if AGENTCORE_AVAILABLE:
    print("🚀 Starting AgentCore Runtime cleanup...")
    try:
        runtime_resource_cleanup()
        print("✅ AgentCore Runtime cleanup completed")
    except Exception as e:
        print(f"⚠️  Error during runtime cleanup: {e}")
else:
    print("ℹ️  Skipping AgentCore Runtime cleanup (utilities not available)")

### Step 3.4: Clean Up AgentCore Security Resources

Remove execution roles, Cognito pools, and authentication resources (if AgentCore labs were completed)

In [ ]:
if AGENTCORE_AVAILABLE:
    print("🛡️  Starting AgentCore Security cleanup...")
    import json
    try:
        # Delete execution role
        print("  🗑️  Deleting AgentCore Runtime execution role...")
        delete_agentcore_runtime_execution_role()
        print("  ✅ Execution role deleted")
        
        # Delete SSM parameter
        print("  🗑️  Deleting SSM parameter...")
        delete_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
        print("  ✅ SSM parameter deleted")
        
        # Clean up Cognito and secrets
        try:
            print("  🗑️  Cleaning up Cognito resources...")
            cs = json.loads(get_customer_support_secret())
            cleanup_cognito_resources(cs['pool_id'])
            print("  ✅ Cognito resources cleaned up")
        except Exception as e:
            print(f"  ℹ️  Cognito cleanup skipped: {e}")
        
        print("  🗑️  Deleting customer support secret...")
        delete_customer_support_secret()
        print("  ✅ Customer support secret deleted")
        
        print("✅ AgentCore Security cleanup completed")
    except Exception as e:
        print(f"⚠️  Error during security cleanup: {e}")
else:
    print("ℹ️  Skipping AgentCore Security cleanup (utilities not available)")

### Step 3.5: Delete S3 Vectors

This is for the chatbot notebooks that Florian modified (when we take them out no longer needed)

In [ ]:
vector_buckets = s3vectors_client.list_vector_buckets()['vectorBuckets']
    
workshop_patterns = [
    'pace-bootcamp-',
    'strands-tutorial-'
]
    
for bucket in vector_buckets:
    bucket_name = bucket['vectorBucketName']
    
    # Check if bucket matches workshop naming patterns
    if any(pattern in bucket_name for pattern in workshop_patterns):
        print(f"Found workshop S3 Vector bucket: {bucket_name}")
        
        # Delete all indexes in the bucket first
        indexes = s3vectors_client.list_indexes(vectorBucketName=bucket_name)['indexes']
        for index in indexes:
            index_name = index['indexName']
            print(f"  -> Deleting vector index: {index_name}")
            s3vectors_client.delete_index(
                vectorBucketName=bucket_name, 
                indexName=index_name
            )

        # Delete the vector bucket
        print(f"  -> Deleting S3 Vector bucket: {bucket_name}")
        s3vectors_client.delete_vector_bucket(vectorBucketName=bucket_name)

print("✓ S3 Vector cleanup completed")

### Step 3.6: Clean Up Knowledge Base Resources

Remove Knowledge Bases (AOSS and S3 Vectors), OpenSearch Serverless collections, and related resources


In [ ]:
print("📚 Starting Knowledge Base cleanup...\n")

try:
    # Add the knowledgebases_and_rag directory to path
    kb_path = os.path.join(os.path.dirname(os.getcwd()), 'knowledgebases_and_rag')
    if os.path.exists(kb_path) and kb_path not in sys.path:
        sys.path.insert(0, kb_path)
    
    # Import and run the cleanup
    from cleanup_kb_resources import KnowledgeBaseCleanup
    
    region = os.environ.get('AWS_REGION', 'us-east-1')
    kb_cleanup = KnowledgeBaseCleanup(region_name=region, dry_run=False)
    kb_cleanup.run_cleanup()
    
    print("\n✅ Knowledge Base cleanup completed")
    
except ImportError:
    print("ℹ️  Knowledge Base cleanup script not found (skipping)")
except Exception as e:
    print(f"⚠️  Error during Knowledge Base cleanup: {e}")


### Step 3.6: Delete Remaining AgentCore Runtimes

Clean up any remaining AgentCore Runtime instances not caught by previous steps

In [ ]:
runtimes = agentcore_control_client.list_agent_runtimes()['agentRuntimes']
for runtime in runtimes:
    runtime_id = runtime['agentRuntimeId']
    status = runtime.get('status', 'UNKNOWN')

    if status == 'DELETING':
        print(f"Skipping AgentCore Runtime {runtime_id} - already deleting")
        continue
        
    print(f"Deleting AgentCore Runtime {runtime_id}...")
    agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=runtime_id
    )

### Step 3.7: Delete Amazon Elastic Container Registry Repositories

In [ ]:
print("Deleting ECR repository...")
repositories = ecr_client.describe_repositories()['repositories']
for repository in repositories:
    name = repository['repositoryName']
    if name.startswith('bedrock-agentcore-mcp_server_agentcore'):
        ecr_client.delete_repository(
            repositoryName=name,
            force=True
        )
print("✓ ECR repository deleted")

### Step 3.8: Delete Secrets Manager Secrets

In [ ]:
# Get list of all secrets
secrets = secrets_client.list_secrets()

# Iterate through secrets and delete those starting with mcp_server
for secret in secrets['SecretList']:
    print(secret)
    secrets_client.delete_secret(
        SecretId=secret['ARN'],
        ForceDeleteWithoutRecovery=True
    )

### Step 3.9: Delete CodeBuild Projects

In [ ]:
projects = codebuild.list_projects()['projects']
for project in projects:
    print(project)
    if project.startswith('bedrock-agentcore-'):
        codebuild.delete_project(name=project)

### Step 3.10: Clean Up Observability Resources

Remove CloudWatch log groups and streams used for agent monitoring

In [ ]:
if AGENTCORE_AVAILABLE:
    print("📊 Starting Observability cleanup...")
    try:
        delete_observability_resources()
        print("✅ Observability cleanup completed")
    except Exception as e:
        print(f"⚠️  Error during observability cleanup: {e}")
else:
    print("ℹ️  Skipping Observability cleanup (utilities not available)")

# Also clean up any bedrock-agentcore log groups
print("\n📊 Cleaning up AgentCore log groups...")
try:
    log_groups = logs_client.describe_log_groups(logGroupNamePrefix='/aws/bedrock-agentcore/')
    for log_group in log_groups.get('logGroups', []):
        log_group_name = log_group['logGroupName']
        print(f"  🗑️  Deleting log group: {log_group_name}")
        logs_client.delete_log_group(logGroupName=log_group_name)
        print(f"  ✅ Deleted: {log_group_name}")
    print("✅ AgentCore log groups cleanup completed")
except Exception as e:
    print(f"⚠️  Error cleaning up log groups: {e}")

### Step 3.11: Clean Up CloudFormation Stacks

Delete the infrastructure stacks created by the AgentCore prereq.sh script (Lambda, DynamoDB, Cognito, IAM roles, etc.)

In [ ]:
if AGENTCORE_AVAILABLE:
    print("☁️  Starting CloudFormation Stack cleanup...")
    
    import time
    from botocore.exceptions import ClientError
    
    cfn_client = boto3.client('cloudformation')
    s3_client = boto3.client('s3')
    
    # Stack names (matching prereq.sh defaults)
    INFRA_STACK_NAME = 'CustomerSupportStackInfra'
    COGNITO_STACK_NAME = 'CustomerSupportStackCognito'
    
    # Get region and account for bucket name
    region = boto3.session.Session().region_name
    account_id = boto3.client('sts').get_caller_identity()['Account']
    lambda_bucket_name = f'customersupport112-{account_id}-{region}'
    kb_bucket_name = f'{account_id}-{region}-kb-data-bucket'
    
    def delete_stack_if_exists(stack_name):
        """Delete a CloudFormation stack if it exists"""
        try:
            # Check if stack exists
            cfn_client.describe_stacks(StackName=stack_name)
            
            print(f"  🗑️  Deleting stack: {stack_name}...")
            cfn_client.delete_stack(StackName=stack_name)
            
            print(f"  ⏳ Waiting for {stack_name} to be deleted...")
            waiter = cfn_client.get_waiter('stack_delete_complete')
            waiter.wait(
                StackName=stack_name,
                WaiterConfig={'Delay': 10, 'MaxAttempts': 120}
            )
            print(f"  ✅ Stack {stack_name} deleted successfully")
            return True
            
        except ClientError as e:
            if 'does not exist' in str(e):
                print(f"  ℹ️  Stack {stack_name} does not exist (already deleted)")
                return True
            else:
                print(f"  ⚠️  Error deleting stack {stack_name}: {e}")
                return False
        except Exception as e:
            print(f"  ⚠️  Unexpected error with stack {stack_name}: {e}")
            return False
    
    def cleanup_s3_bucket(bucket_name):
        """Empty and optionally delete the S3 bucket"""
        try:
            # Check if bucket exists
            s3_client.head_bucket(Bucket=bucket_name)
            
            print(f"  🗑️  Emptying S3 bucket: {bucket_name}...")
            
            # List and delete all objects
            paginator = s3_client.get_paginator('list_objects_v2')
            for page in paginator.paginate(Bucket=bucket_name):
                if 'Contents' in page:
                    objects = [{'Key': obj['Key']} for obj in page['Contents']]
                    s3_client.delete_objects(
                        Bucket=bucket_name,
                        Delete={'Objects': objects}
                    )
                    print(f"    Deleted {len(objects)} objects")
            
            print(f"  ✅ S3 bucket {bucket_name} emptied")
            
            # Optionally delete the bucket
            print(f"  🗑️  Deleting S3 bucket: {bucket_name}...")
            s3_client.delete_bucket(Bucket=bucket_name)
            print(f"  ✅ S3 bucket {bucket_name} deleted")
            return True
            
        except ClientError as e:
            if e.response['Error']['Code'] == '404':
                print(f"  ℹ️  S3 bucket {bucket_name} does not exist (already deleted)")
                return True
            else:
                print(f"  ⚠️  Error with S3 bucket {bucket_name}: {e}")
                return False
        except Exception as e:
            print(f"  ⚠️  Unexpected error with S3 bucket: {e}")
            return False
    
    try:
        # IMPORTANT: Empty S3 buckets BEFORE deleting stacks
        # CloudFormation cannot delete non-empty S3 buckets
        
        # Clean up Lambda deployment bucket
        print("  📦 Cleaning up Lambda deployment bucket...")
        cleanup_s3_bucket(lambda_bucket_name)
        
        # Now delete CloudFormation stacks (buckets are empty now)
        # Delete Infrastructure Stack (contains Lambda, DynamoDB, IAM roles)
        delete_stack_if_exists(INFRA_STACK_NAME)
        
        # Delete Cognito Stack
        delete_stack_if_exists(COGNITO_STACK_NAME)
        
        # Clean up local zip file
        local_zip = 'lambda.zip'
        if os.path.exists(local_zip):
            os.remove(local_zip)
            print(f"  ✅ Deleted local file: {local_zip}")
        
        print("✅ CloudFormation Stack cleanup completed")

        
    except Exception as e:
        print(f"⚠️  Error during CloudFormation cleanup: {e}")
else:
    print("ℹ️  Skipping CloudFormation Stack cleanup (AgentCore utilities not available)")

### Step 3.12: Clean Up Local Files

Remove generated configuration and code files from the AgentCore labs directory

In [ ]:
if AGENTCORE_AVAILABLE:
    print("📁 Starting Local Files cleanup...")
    try:
        # Change to AgentCore directory if it exists
        original_dir = os.getcwd()
        if os.path.exists(agentcore_path):
            os.chdir(agentcore_path)
            local_file_cleanup()
            os.chdir(original_dir)
            print("✅ Local files cleanup completed")
        else:
            print("ℹ️  AgentCore directory not found, skipping local file cleanup")
    except Exception as e:
        print(f"⚠️  Error during local files cleanup: {e}")
        os.chdir(original_dir)
else:
    print("ℹ️  Skipping Local Files cleanup (utilities not available)")

### Step 3.13: Clean Up Strands Agents Resources

Remove resources created by Strands Agents notebooks (Knowledge Base, OpenSearch Serverless, DynamoDB, S3)

In [ ]:
# Determine the correct path to strands_agents
# If we're in labs/cleanup, go up one level
current_dir = os.getcwd()
if current_dir.endswith('cleanup'):
    strands_agents_path = '../strands_agents/02-connecting-with-aws-services'
else:
    strands_agents_path = 'labs/strands_agents/02-connecting-with-aws-services'

# Resolve to absolute path
strands_agents_path = os.path.abspath(strands_agents_path)

print(f"Looking for Strands Agents at: {strands_agents_path}")

if os.path.exists(strands_agents_path):
    print("🤖 Starting Strands Agents cleanup...")
    
    try:
        # Change to the notebook directory
        original_dir = os.getcwd()
        os.chdir(strands_agents_path)
        
        # Run the cleanup script
        import subprocess
        print("  🗑️  Running Strands Agents cleanup script...")
        result = subprocess.run(
            ['bash', 'cleanup.sh'],
            capture_output=True,
            text=True,
            timeout=300  # 5 minute timeout
        )
        
        # Print output
        if result.stdout:
            print(result.stdout)
        
        if result.returncode == 0:
            print("✅ Strands Agents resources cleaned up successfully")
        else:
            print(f"⚠️  Cleanup script completed with warnings")
            if result.stderr:
                print(f"  Errors: {result.stderr}")
        
        os.chdir(original_dir)
        
    except subprocess.TimeoutExpired:
        print("⚠️  Cleanup script timed out after 5 minutes")
        os.chdir(original_dir)
    except Exception as e:
        print(f"⚠️  Error during Strands Agents cleanup: {e}")
        try:
            os.chdir(original_dir)
        except:
            pass
else:
    print("ℹ️  Strands Agents directory not found, skipping cleanup")

### Step 3.14: Clean Up Bedrock Data Automation Resources

Remove BDA projects, blueprints, and S3 buckets created during the workshop

In [ ]:
print("📄 Starting Bedrock Data Automation cleanup...")

try:
    # Initialize BDA clients
    bda_client = boto3.client('bedrock-data-automation')
    s3_client = boto3.client('s3')
    
    # Get account details for bucket naming
    region = boto3.session.Session().region_name
    account_id = boto3.client('sts').get_caller_identity()['Account']
    bda_bucket = f'pace-bootcamp-bda-bucket-{account_id}-{region}'
    
    print(f"  🪣 BDA S3 bucket: {bda_bucket}")
    
    # 1. Delete BDA Projects
    print("  📋 Listing BDA projects...")
    try:
        projects_response = bda_client.list_data_automation_projects()
        projects = projects_response.get('projects', [])
        
        workshop_project_patterns = [
            'advanced_standard_output_project',
            'document-custom-output-multiple-blueprints',
            'pace-bootcamp',
            'workshop'
        ]
        
        deleted_projects = 0
        for project in projects:
            project_name = project.get('projectName', '')
            project_arn = project.get('projectArn', '')
            
            # Check if project matches workshop patterns
            is_workshop_project = any(
                pattern in project_name.lower() 
                for pattern in workshop_project_patterns
            )
            
            if is_workshop_project:
                print(f"    🗑️  Deleting BDA project: {project_name}")
                try:
                    bda_client.delete_data_automation_project(projectArn=project_arn)
                    deleted_projects += 1
                    print(f"    ✅ Project {project_name} deleted")
                except Exception as e:
                    print(f"    ⚠️  Error deleting project {project_name}: {e}")
            else:
                print(f"    ⏭️  Skipping non-workshop project: {project_name}")
        
        if deleted_projects > 0:
            print(f"  ✅ Deleted {deleted_projects} BDA project(s)")
        else:
            print("  ℹ️  No workshop BDA projects found to delete")
            
    except Exception as e:
        print(f"  ⚠️  Error listing/deleting BDA projects: {e}")
    
    # 2. Delete Custom Blueprints
    print("  📐 Listing custom blueprints...")
    try:
        blueprints_response = bda_client.list_blueprints()
        blueprints = blueprints_response.get('blueprints', [])
        
        workshop_blueprint_patterns = [
            'claim-form',
            'hospital-discharge-report',
            'medical-transcription',
            'lab-reports',
            'explanation-of-benefits',
            'pace-bootcamp',
            'workshop'
        ]
        
        deleted_blueprints = 0
        for blueprint in blueprints:
            blueprint_name = blueprint.get('blueprintName', '')
            blueprint_arn = blueprint.get('blueprintArn', '')
            
            # Skip AWS-managed blueprints (they start with 'arn:aws:bedrock:region:aws:')
            if ':aws:blueprint/' in blueprint_arn:
                continue
            
            # Check if blueprint matches workshop patterns
            is_workshop_blueprint = any(
                pattern in blueprint_name.lower() 
                for pattern in workshop_blueprint_patterns
            )
            
            if is_workshop_blueprint:
                print(f"    🗑️  Deleting custom blueprint: {blueprint_name}")
                try:
                    bda_client.delete_blueprint(blueprintArn=blueprint_arn)
                    deleted_blueprints += 1
                    print(f"    ✅ Blueprint {blueprint_name} deleted")
                except Exception as e:
                    if 'ValidationException' in str(e) and 'referenced by some project' in str(e):
                        print(f"    ⚠️  Blueprint {blueprint_name} still referenced by project, will be cleaned up with project deletion")
                    else:
                        print(f"    ⚠️  Error deleting blueprint {blueprint_name}: {e}")
            else:
                print(f"    ⏭️  Skipping non-workshop blueprint: {blueprint_name}")
        
        if deleted_blueprints > 0:
            print(f"  ✅ Deleted {deleted_blueprints} custom blueprint(s)")
        else:
            print("  ℹ️  No workshop custom blueprints found to delete")
            
    except Exception as e:
        print(f"  ⚠️  Error listing/deleting custom blueprints: {e}")

    print("✅ Bedrock Data Automation cleanup completed")
    
except Exception as e:
    print(f"⚠️  Error during BDA cleanup: {e}")

### Step 3.15: Clean Up Bedrock Guardrails

Remove Bedrock Guardrails created during the workshop

In [ ]:
print("🛡️  Starting Bedrock Guardrails cleanup...")

try:
    bedrock_client = boto3.client('bedrock')
    
    # List all guardrails
    print("  📋 Listing Bedrock Guardrails...")
    guardrails_response = bedrock_client.list_guardrails(maxResults=100)
    guardrails = guardrails_response.get('guardrails', [])
    
    if not guardrails:
        print("  ℹ️  No guardrails found")
    else:
       
        
        deleted_count = 0
        for guardrail in guardrails:
            guardrail_id = guardrail['id']
            guardrail_name = guardrail.get('name', 'Unknown')
            
            print(f"  🗑️  Deleting guardrail: {guardrail_name} ({guardrail_id})")
            try:
                bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
                deleted_count += 1
                print(f"  ✅ Guardrail {guardrail_name} deleted")
            except ClientError as e:
                if 'ResourceNotFoundException' in str(e):
                    print(f"  ℹ️  Guardrail {guardrail_name} already deleted")
                else:
                    print(f"  ⚠️  Error deleting guardrail {guardrail_name}: {e}")
            except Exception as e:
                print(f"  ⚠️  Error deleting guardrail {guardrail_name}: {e}")
            
        
        if deleted_count > 0:
            print(f"✅ Deleted {deleted_count} Bedrock Guardrail(s)")
        else:
            print("ℹ️  No workshop guardrails found to delete")
    
except ClientError as e:
    if 'AccessDeniedException' in str(e):
        print("⚠️  Access denied to Bedrock Guardrails. Skipping...")
    else:
        print(f"⚠️  Error during Guardrails cleanup: {e}")
except Exception as e:
    print(f"⚠️  Error during Guardrails cleanup: {e}")

print("✅ Bedrock Guardrails cleanup completed")

## 🎉 Cleanup Complete!

All PACE Bootcamp resources have been cleaned up. Here's a summary of what was removed:

In [ ]:
print("\n" + "=" * 70)
print("🧹 CLEANUP COMPLETED SUCCESSFULLY! 🧹")
print("=" * 70)
print()
print("📋 Resources cleaned up:")
print("  🏷️  Tagged Resources: All resources with app:pace_bootcamp tag")
print("  🧠 Memory: AgentCore Memory resources and data")
print("  ⚙️  Gateway: AgentCore Gateway targets and endpoints")
print("  🚀 Runtime: Agent runtime instances")
print("  🐳 ECR: Container repositories and images")
print("  📦 S3 Vectors: Vector buckets and indexes")
print("  🔐 Secrets: Secrets Manager secrets")
print("  🔨 CodeBuild: Build projects")
print("  🛡️  Security: IAM roles and Cognito pools")
print("  ☁️  CloudFormation: Infrastructure and Cognito stacks")
print("  🪣 S3: Lambda deployment bucket and contents")
print("  📊 Observability: CloudWatch logs")
print("  🤖 Strands Agents: Knowledge Base, OpenSearch, DynamoDB")
print("  🛡️  Guardrails: Bedrock Guardrails")
print("  📁 Files: Local configuration files")
print()
print("✨ Your AWS account is now clean and ready for new experiments!")
print("\nThank you for completing the PACE Bootcamp! 🚀")